# DeepAgents 学习教程

## 教程内容

1. **基础示例**：使用 DeepAgents 创建简单的研究助手
2. **高级示例**：使用技能和检查点创建更复杂的代理
3. **存储后端**：使用 StoreBackend 管理代理状态

## 前提条件

- 已安装必要的依赖包
- 已配置好 `.env` 文件（包含 DASHSCOPE_API_KEY 和 TAVILY_API_KEY）
- 已创建 `util.py` 文件，包含 `tavily_client` 和 `model` 的初始化

In [ ]:
# 导入必要的库
from typing import Literal
from urllib.request import urlopen
from deepagents import create_deep_agent
from langgraph.checkpoint.memory import MemorySaver
from deepagents.backends import StoreBackend
from langgraph.store.memory import InMemoryStore
from util import tavily_client, model

## 1. 基础示例：创建研究助手

本示例演示如何创建一个简单的研究助手，使用 Tavily 搜索引擎进行网络搜索。

In [ ]:
# 定义网络搜索工具
def internet_search(
        query: str,
        max_results: int = 5,
        topic: Literal['general', 'news', 'finance'] = 'general',
        include_raw_content: bool = False,
):
    """Run a web search"""
    return tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )

In [ ]:
# 创建研究助手代理
research_instructions = """\
You are an expert researcher. Your job is to conduct \n
thorough research, and then write a polished report. \n
"""\

agent = create_deep_agent(
    model=model,
    system_prompt=research_instructions,
    tools=[internet_search]
)

# 测试代理
result = agent.stream({"messages": [{"role": "user", "content": "What is deep learning?"}]})
for e in result:
    print(e)

## 2. 高级示例：使用技能和检查点

本示例演示如何使用技能（Skills）和检查点（Checkpointer）创建更复杂的代理。技能是 DeepAgents 中的一种机制，允许代理使用预定义的功能和知识。

In [ ]:
# 创建检查点（用于保存代理状态）
checkpointer = MemorySaver()

# 下载技能文件
skill_url = "https://raw.githubusercontent.com/langchain-ai/deepagentsjs/refs/heads/main/examples/skills/langgraph-docs/SKILL.md"
with urlopen(skill_url) as response:
    skill_content = response.read().decode('utf-8')

# 准备技能文件
skills_files = {
    "/skills/langgraph-docs/SKILL.md": skill_content
}

# 创建带技能的代理
agent_with_skills = create_deep_agent(
    model=model,
    skills=["./skills/"],
    checkpointer=checkpointer,
)

# 测试代理
result = agent_with_skills.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What is langgraph?",
            }
        ],
        # 为默认的 StateBackend 提供虚拟文件系统
        "files": skills_files
    },
    config={"configurable": {"thread_id": "12345"}},
)

# 打印结果
print(result["messages"][-1]["content"])

## 3. 存储后端：使用 StoreBackend

本示例演示如何使用 StoreBackend 管理代理状态，这是一种更灵活的状态管理方式。

In [ ]:
# 创建内存存储
store = InMemoryStore()

# 下载技能文件
skill_url = "https://raw.githubusercontent.com/langchain-ai/deepagentsjs/refs/heads/main/examples/skills/langgraph-docs/SKILL.md"
with urlopen(skill_url) as response:
    skill_content = response.read().decode('utf-8')

# 将技能文件存储到内存存储中
store.put(
    namespace=("filesystem",),
    key="/skills/langgraph-docs/SKILL.md",
    value=skill_content
)

# 创建使用 StoreBackend 的代理
agent_with_store = create_deep_agent(
    backend=(lambda rt: StoreBackend(rt)),
    store=store,
    skills=["./skills/"]
)

# 测试代理
result = agent_with_store.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What is langgraph?",
            }
        ]
    },
    config={"configurable": {"thread_id": "12345"}},
)

# 打印结果
print(result["messages"][-1]["content"])

## 4. 总结与扩展

通过本教程，您学习了如何使用 DeepAgents 创建不同类型的代理：

### 基础概念

1. **模型**：使用 `model` 参数指定要使用的语言模型（本教程使用 Qwen 模型）
2. **工具**：通过 `tools` 参数为代理提供外部工具（如网络搜索）
3. **系统提示**：使用 `system_prompt` 定义代理的角色和行为

### 高级特性

1. **技能**：通过 `skills` 参数为代理提供预定义的功能和知识
2. **检查点**：使用 `checkpointer` 保存和恢复代理状态
3. **存储后端**：使用 `StoreBackend` 更灵活地管理代理状态

### 扩展建议

1. **添加更多工具**：为代理添加其他有用的工具，如文件操作、数据库查询等
2. **自定义技能**：创建自己的技能文件，为代理提供特定领域的知识
3. **使用持久存储**：将 `InMemoryStore` 替换为持久存储（如数据库）
4. **多代理协作**：创建多个代理并让它们协作完成复杂任务
5. **添加监控和日志**：为代理添加监控和日志功能，以便更好地理解其行为

### 故障排除

如果遇到问题，请检查：

1. `.env` 文件是否正确配置
2. `util.py` 文件是否正确初始化了 `tavily_client` 和 `model`
3. 网络连接是否正常（特别是下载技能文件时）
4. 依赖包是否已正确安装

In [ ]:
# 测试所有代理是否正常工作
print("=== 测试基础代理 ===")
try:
    result = agent.stream({"messages": [{"role": "user", "content": "What is AI?"}]})
    print("基础代理测试成功！")
except Exception as e:
    print(f"基础代理测试失败: {e}")

print("=== 测试带技能的代理 ===")
try:
    result = agent_with_skills.invoke(
        {"messages": [{"role": "user", "content": "What is langgraph?"}],
        "files": skills_files
    },
    config={"configurable": {"thread_id": "67890"}},
    )
    print("带技能的代理测试成功！")
except Exception as e:
    print(f"带技能的代理测试失败: {e}")

print("=== 测试带存储后端的代理 ===")
try:
    result = agent_with_store.invoke(
        {"messages": [{"role": "user", "content": "What is langgraph?"}]},
        config={"configurable": {"thread_id": "67890"}},
    )
    print("带存储后端的代理测试成功！")
except Exception as e:
    print(f"带存储后端的代理测试失败: {e}")
